# Fit emimTF2N data

EoS: SRK

Parameter T dependence: Order 3 Polynomial

N (total fitting parameters): 8

**Finds an optimal solution, but can't get it to work when plotting - the control volume fails to initialize

## Import Functions

In [1]:
import idaes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

import sys
sys.path.append('../../../')

from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.opt import TerminationCondition, SolverStatus

from idaes.core import FlowsheetBlock
import idaes.logger as idaeslog
# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
# Import unit models from the model library
from idaes.generic_models.unit_models import Flash
# Import degrees of freedom tool
from idaes.core.util.model_statistics import degrees_of_freedom

# parmest (binary_param2)
from bip_fitting_functions_SRK import polynomial

import scipy.stats as stats

## Load Data

In [2]:
data_full = pd.read_csv('../r125_emimtf2n_full.csv')
data_subset = pd.read_csv('../r125_emimtf2n_subset.csv')
data_extended = pd.read_csv('../r125-emimtf2n-extended.csv')

## Load Configuration + Check DOF + Load Parameters

In [3]:
lhs_results1 = pd.read_csv('../Data/Fits/LHS_Fits/SRK_polyTdep_LHS.csv',index_col=0)
lhs_results1 = lhs_results1[['sc_param1','sc_param2','sc_param3','sc_param4','sc_param5','sc_param6','sc_param7','sc_param8','SSR']]

lhs_results2 = pd.read_csv('../Data/Fits/LHS_Fits/SRK_polyTdep_LHS_fromsc.csv',index_col=0)
lhs_results2 = lhs_results2[['sc_param1','sc_param2','sc_param3','sc_param4','sc_param5','sc_param6','sc_param7','sc_param8','SSR']]

lhs_results3 = pd.read_csv('../Data/Fits/LHS_Fits/SRK_polyTdep_LHS_fromsc11.csv',index_col=0)
lhs_results3 = lhs_results3[['sc_param1','sc_param2','sc_param3','sc_param4','sc_param5','sc_param6','sc_param7','sc_param8','SSR']]

lhs_results4 = pd.read_csv('../Data/Fits/LHS_Fits/SRK_polyTdep_LHS_fromsc227.csv',index_col=0)
lhs_results4 = lhs_results4[['sc_param1','sc_param2','sc_param3','sc_param4','sc_param5','sc_param6','sc_param7','sc_param8','SSR']]

lhs_results5 = pd.read_csv('Data/Init_Final/LHS_Output/SRK_polyTdep_LHS_final.csv',index_col=0)
lhs_results5 = lhs_results5[['sc_param1','sc_param2','sc_param3','sc_param4','sc_param5','sc_param6','sc_param7','sc_param8','SSR']]

lhs_results6 = pd.read_csv('Data/Init_Final/LHS_Output/SRK_polyTdep_LHS_fromquad_final.csv',index_col=0)
lhs_results6 = lhs_results6[['sc_param1','sc_param2','sc_param3','sc_param4','sc_param5','sc_param6','sc_param7','sc_param8','SSR']]

lhs_results = pd.concat([lhs_results1,lhs_results2,lhs_results3,lhs_results4,lhs_results5,lhs_results6],axis=0).reset_index(drop=True)

In [4]:
lhs_results.dropna(axis=0)

,sc_param1,sc_param2,sc_param3,sc_param4,sc_param5,sc_param6,sc_param7,sc_param8,SSR
691,-0.932453,-0.459271,-0.911759,-0.513797,0.692297,0.936531,0.233765,0.016461,13.029285
724,-0.932453,-0.459271,-0.911759,-0.513797,0.692297,0.936531,0.136351,0.031701,13.059942
906,0.140683,-0.506367,-0.463518,0.552634,0.371398,-0.011327,-0.080361,-0.074367,13.059942
970,0.140683,-0.506367,-0.463518,0.552634,0.371398,-0.011327,-0.165507,-0.154970,2566.781524
1166,-1.951156,-1.677026,-0.610838,1.175293,-1.071340,0.670624,1.070138,-0.144267,375.356890
6501,0.711122,0.175527,-0.938228,-1.881777,-1.441844,1.679764,0.797194,0.093325,13.029285
7920,0.711122,0.175527,-0.938228,-1.881777,-1.441844,1.679764,0.137375,0.015709,2761.640204
9564,0.711122,0.175527,-0.938228,-1.881777,-1.441844,1.679764,-0.049402,-0.026052,13.029285
9902,0.711122,0.175527,-0.938228,-1.881777,-1.441844,1.679764,-0.083052,-0.000258,1010.191466
9949,0.711122,0.175527,-0.938228,-1.881777,-1.441844,1.679764,0.237887,0.010243,2110.822436


In [5]:
lhs_best = lhs_results[lhs_results.SSR == lhs_results.SSR.min()]
lhs_best

,sc_param1,sc_param2,sc_param3,sc_param4,sc_param5,sc_param6,sc_param7,sc_param8,SSR
10701,0.711122,0.175527,-0.938228,-1.881777,-1.441844,1.679764,0.0832,0.007522,10.972701


In [6]:
sc_param1 = lhs_best.at[10701, 'sc_param1']
sc_param2 = lhs_best.at[10701, 'sc_param2']
sc_param3 = lhs_best.at[10701, 'sc_param3']
sc_param4 = lhs_best.at[10701, 'sc_param4']
sc_param5 = lhs_best.at[10701, 'sc_param5']
sc_param6 = lhs_best.at[10701, 'sc_param6']
sc_param7 = lhs_best.at[10701, 'sc_param7']
sc_param8 = lhs_best.at[10701, 'sc_param8']

In [7]:
from hfc125_emimtf2n_SRK_polynomial import configuration 

In [8]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
m.fs.properties = GenericParameterBlock(default=configuration)
m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})
DOF_initial = degrees_of_freedom(m)
print('The initial DOF is {0}'.format(DOF_initial))

The initial DOF is 7


## Fit Parameters

In [9]:
parameters, obj_value, a = polynomial(data_subset, configuration, 'R125', 'emimTf2N', "x_R125", "x_emimTf2N", 
        init_temp =  283.1, init_press =   399300 , init_x_c1 =    0.448, init_x_c2 = 0.552,
        init_kappa_2_1A = sc_param1, init_kappa_1_2A = sc_param2,
        init_kappa_2_1B = sc_param3, init_kappa_1_2B = sc_param4,
        init_kappa_2_1C = sc_param5, init_kappa_1_2C = sc_param6,
        init_kappa_2_1D = sc_param7, init_kappa_1_2D = sc_param8,
        eps = 0.1, scaling_fac = 1e-7, 
        filename='Data/Fits/Ipopt_Output/SRK_polyTdep_final.txt')

2022-10-14 12:32:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:32:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:32:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:32:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:32:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:32:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:32:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:32:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:32:38 [INFO] idaes

/Users/bridgettebefort/opt/anaconda3/envs/idaes-pse-dev/lib/python3.9/site-packages/pyomo/contrib/interior_point/interface.py:418: RuntimeWarning: divide by zero encountered in true_divide
  data = (duals_primals_lb/(primals - self._nlp.primals_lb()) +


In [10]:
print('Objective (SSE):',obj_value)

Objective (SSE): 10.972701417726299


In [11]:
print('Parameters:',parameters)

Parameters: fs.properties.SRK_kappa_A[emimTf2N,R125]    -4.316561
fs.properties.SRK_kappa_A[R125,emimTf2N]    -2.685470
fs.properties.SRK_kappa_B[emimTf2N,R125]    10.862388
fs.properties.SRK_kappa_B[R125,emimTf2N]     7.606709
fs.properties.SRK_kappa_C[emimTf2N,R125]    -8.885760
fs.properties.SRK_kappa_C[R125,emimTf2N]    -7.246046
fs.properties.SRK_kappa_D[emimTf2N,R125]     2.278946
fs.properties.SRK_kappa_D[R125,emimTf2N]     2.295020
dtype: float64


In [12]:
params2save = np.zeros(8)
params2save[0] = parameters[1]
params2save[1] = parameters[0]
params2save[2] = parameters[3]
params2save[3] = parameters[2]
params2save[4] = parameters[5]
params2save[5] = parameters[4]
params2save[6] = parameters[7]
params2save[7] = parameters[6]

In [13]:
np.savetxt('Data/Fits/SRK_params_polyTdep.csv',params2save,delimiter=',')

In [14]:
cov = np.array(a)
print('Covariance Matrix:')
print(cov)

Covariance Matrix:
[[  41.65798262   19.68564752  -98.24618754  -55.08148842   72.21889234
    51.15987129  -15.70706953  -15.77333478]
 [  19.68564752   11.16225055  -47.95344245  -31.32110861   37.25882239
    29.1885995    -9.02506729   -9.03465746]
 [ -98.24618754  -47.95344245  233.03361751  134.29588131 -172.9654092
  -124.85665449   38.35515164   38.53650941]
 [ -55.08148843  -31.32110861  134.29588132   87.91436272 -104.46810721
   -81.95466363   25.34865192   25.37518063]
 [  72.21889235   37.25882239 -172.9654092  -104.46810721  130.51634735
    97.25472536  -29.89729745  -30.0623489 ]
 [  51.1598713    29.18859951 -124.85665449  -81.95466363   97.25472536
    76.42344433  -23.64576828  -23.67018074]
 [ -15.70706953   -9.02506729   38.35515164   25.34865192  -29.89729745
   -23.64576828    7.27618264    7.32613356]
 [ -15.77333478   -9.03465746   38.53650941   25.37518063  -30.0623489
   -23.67018074    7.32613356    7.33360704]]


In [15]:
np.savetxt('Data/Covariance/SRK_Cov_polyTdep.csv',cov,delimiter=',')

## Apply Parameters + Plot

In [ ]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [ ]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.properties = GenericParameterBlock(default=configuration)

m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m.fs.F101.inlet.flow_mol.fix(1)
m.fs.F101.inlet.temperature.fix(283.15)
m.fs.F101.inlet.pressure.fix(100000)
m.fs.F101.inlet.mole_frac_comp[0,'R125'].fix(0.1)

m.fs.properties.SRK_kappa_A['R125','emimTf2N'] = parameters['fs.properties.SRK_kappa_A[R125,emimTf2N]']
m.fs.properties.SRK_kappa_A['emimTf2N','R125'] = parameters['fs.properties.SRK_kappa_A[emimTf2N,R125]']
m.fs.properties.SRK_kappa_B['R125','emimTf2N'] = parameters['fs.properties.SRK_kappa_B[R125,emimTf2N]']
m.fs.properties.SRK_kappa_B['emimTf2N','R125'] = parameters['fs.properties.SRK_kappa_B[emimTf2N,R125]']
m.fs.properties.SRK_kappa_C['R125','emimTf2N'] = parameters['fs.properties.SRK_kappa_C[R125,emimTf2N]']
m.fs.properties.SRK_kappa_C['emimTf2N','R125'] = parameters['fs.properties.SRK_kappa_C[emimTf2N,R125]']
m.fs.properties.SRK_kappa_D['R125','emimTf2N'] = parameters['fs.properties.SRK_kappa_D[R125,emimTf2N]']
m.fs.properties.SRK_kappa_D['emimTf2N','R125'] = parameters['fs.properties.SRK_kappa_D[emimTf2N,R125]']

# Initialize the flash unit
m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)

# Fix the state variables on the state block
# m.fs.F101.inlet.pressure.unfix()
m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].unfix()

R125_x = np.zeros((len(data_extended)))
P = np.zeros((len(data_extended)))
T = np.zeros((len(data_extended)))

m.fs.liq = Param(mutable=True,default=0.040)
m.fs.liquid = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, "emimTf2N"] == m.fs.liq)

for i in range(len(data_extended)):
    print('i:',i)
    print('Try x_IL:',data_extended["x_emimTf2N"].iloc[i])
    m.fs.liq = data_extended["x_emimTf2N"].iloc[i]
    m.fs.F101.inlet.temperature.fix(float(data_extended["temperature"].iloc[i]))
    m.fs.F101.inlet.pressure.fix(float(data_extended["pressure"].iloc[i]))
    m.fs.F101.inlet.mole_frac_comp[0,'R125'].fix(float(data_extended["x_R125"].iloc[i])+0.1)
    m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].fix(float(1-(data_extended["x_R125"].iloc[i]+0.1)))
    m.fs.F101.vap_outlet.temperature.fix(float(data_extended["temperature"].iloc[i]))
    
    try:
        m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
        
    except:
        print('Initialization Error')
        
    try:

        status = solver.solve(m, tee = False)
        
        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):
            print('Feasible/Optimal Solution')
            R125_x[i] = value(m.fs.F101.liq_outlet.mole_frac_comp[0,'R125']) 
            P[i] = value(m.fs.F101.vap_outlet.pressure[0])
            T[i] = data_extended["temperature"].iloc[i]
        else:
            print('Infeasible Solution')
            R125_x[i] = np.nan
            P[i] = np.nan
            T[i] = data_extended["temperature"].iloc[i]
            
    except ValueError:
        R125_x[i] = np.nan
        P[i] = np.nan
        T[i] = data_extended["temperature"].iloc[i]
        print('Solver Error')

In [ ]:
fitted_data = pd.DataFrame(data=[R125_x,P/1e6,T])
fitted_data = fitted_data.T
fitted_data.columns = ['R125_x','P, MPa','T, K']
fitted_data.to_csv('Data/Fits/SRK_FittedData_polyTdep.csv')

In [ ]:
plt.plot(fitted_data.where(fitted_data['T, K']==283.1)['R125_x'],fitted_data.where(fitted_data['T, K']==283.1)['P, MPa'],"b-",label='T=283 K')
plt.plot(fitted_data.where(fitted_data['T, K']==298.1)['R125_x'],fitted_data.where(fitted_data['T, K']==298.1)['P, MPa'],"r-",label='T=298 K')
plt.plot(fitted_data.where(fitted_data['T, K']==323.1)['R125_x'],fitted_data.where(fitted_data['T, K']==323.1)['P, MPa'],"c-",label='T=323 K')
plt.plot(fitted_data.where(fitted_data['T, K']==348.1)['R125_x'],fitted_data.where(fitted_data['T, K']==348.1)['P, MPa'],"m-",label='T=348 K')
plt.plot(data_full["x_R125"],data_full["pressure"]/1e6,"*",color='orange',label="data")  
plt.title('Isotherm R125-[emim][TF2N]')
plt.ylabel('Pressure (MPa)')
plt.xlabel('x R125')
plt.grid(True)
plt.ylim(0,1.110000)
plt.legend()
plt.show()

### Print Eigvals, Eigvectors

An identifiability analysis can be performed with these as well as a FIM because the covariance matrix is the inverse of the FIM. That means that if there are any eigvals > 0, their eigvecs indicate directions of un-identifiability.

In [ ]:
eigvals, eigvecs = np.linalg.eig(cov)

In [ ]:
eigvals

In [ ]:
eigvecs

In [ ]:
np.savetxt('Data/EigenAnalysis/SRK_vals_polyTdep.csv',eigvals,delimiter=',')
np.savetxt('Data/EigenAnalysis/SRK_vecs_polyTdep.csv',eigvecs,delimiter=',')

In [ ]:
FIM = np.linalg.inv(cov) 

#A_opt
FIM_trace = np.trace(FIM)
cov_trace = np.trace(cov)

#D_opt
FIM_det = np.linalg.det(FIM)

#E_opt
eigvals_FIM, eigvecs_FIM = np.linalg.eig(FIM)
eigvals_FIM_min = eigvals_FIM.min()
eigvals_FIM_max = eigvals_FIM.max()

np.savetxt('Data/EigenAnalysis/SRK_FIMtrdet_polyTdep.csv',[cov_trace,FIM_trace,FIM_det,eigvals_FIM_min,eigvals_FIM_max])
np.savetxt('Data/EigenAnalysis/SRK_FIMvals_polyTdep.csv',eigvals_FIM,delimiter=',')
np.savetxt('Data/EigenAnalysis/SRK_FIMvecs_polyTdep.csv',eigvecs_FIM,delimiter=',')

## AIC Value

See: https://doi.org/10.1080/21642583.2018.1496042

AIC(k) = Nln(MSE(k))+2k

N: number of data points

MSE(k): mean squared error of the model with k parameters

k: number of parameters

MSE(k) = SSE(k)/N

SSE: sum of squared error of the model with k parameters

In [ ]:
#number of datapoints
N = len(data_subset)
#MSE(k)
MSE = obj_value*1E-2/N
#number of parameters
k = 8

In [ ]:
AIC = N*np.log(MSE) + 2*k
AIC

In [ ]:
AIC_data = pd.DataFrame(data=[N,MSE,k,obj_value,AIC])
AIC_data = AIC_data.T
AIC_data.columns = ['N','MSE','k','Objective, Pa^2','AIC']
AIC_data.to_csv('Data/AIC/SRK_AIC_polyTdep.csv')